In [1]:
import glob
import os

import numpy as np
import pandas as pd

In [2]:
dirs = glob.glob("/home/cqql/Seafile/thesis/16-gestures/recordings/*")

In [3]:
events = [pd.read_csv(os.path.join(d, "events.csv")) for d in dirs]
labels = [pd.read_csv(os.path.join(d, "labels.csv")) for d in dirs]

In [4]:
overall_time = sum(e["timestamp"].max() - e["timestamp"].min() for e in events)

print("Total in time in minutes: {}".format(overall_time / 10**6 / 60))

Total in time in minutes: 40.2542592


In [5]:
overall_events = sum(len(e) for e in events)

print("Total number of events: {}".format(overall_events))

Total number of events: 84465277


In [6]:
time = {}
num_events = {}

for e, l in zip(events, labels):
    for start, end, name in l.itertuples(index=False):
        if not name in time:
            time[name] = []
            
        if not name in num_events:
            num_events[name] = []
            
        ts = e["timestamp"].values
        time[name].append(end - start)
        num_events[name].append(np.count_nonzero((ts >= start) & (ts <= end)))
        
time = {k: np.array(t) / 10**6 for k, t in time.items()}
num_events = {k: np.array(e) for k, e in num_events.items()}

In [7]:
num_instances = {k: len(t) for k, t in time.items()}
total_time = {k: sum(t) for k, t in time.items()}
total_events = {k: sum(e) for k, e in num_events.items()}
avg_time = {k: np.mean(t) for k, t in time.items()}
avg_events = {k: np.mean(e) for k, e in num_events.items()}
std_time = {k: np.std(t) for k, t in time.items()}
std_events = {k: np.std(t) for k, t in num_events.items()}

In [8]:
blank_time = overall_time / 10**6 - sum(total_time.values())
blank_events = overall_events - sum(total_events.values())

blank_time / 60, blank_events

(22.176134650000002, 28865096)

In [9]:
df = pd.DataFrame(index=sorted(time.keys()))
df["#inst"] = pd.Series(num_instances)
df["total_time"] = pd.Series(total_time)
df["avg_time"] = pd.Series(avg_time)
df["std_time"] = pd.Series(std_time)
df["total_ev"] = pd.Series(total_events)
df["avg_ev"] = pd.Series(avg_events)
df["std_ev"] = pd.Series(std_events)



df.loc["<blank>"] = [np.nan, blank_time, np.nan, np.nan, blank_events, np.nan, np.nan]

In [10]:
df

,#inst,total_time,avg_time,std_time,total_ev,avg_ev,std_ev
beckoning,41.0,66.763535,1.628379,0.229014,2949776.0,71945.756098,29515.667067
finger-snap,40.0,59.443526,1.486088,0.327845,2203881.0,55097.025000,19777.529730
ok,40.0,63.322015,1.583050,0.195415,3054306.0,76357.650000,18290.085635
push-hand-down,41.0,77.488926,1.889974,0.255861,4000518.0,97573.609756,33154.508194
push-hand-left,40.0,74.389878,1.859747,0.417886,3859846.0,96496.150000,32781.624712
push-hand-right,39.0,75.817942,1.944050,0.267268,5550913.0,142331.102564,57933.151998
push-hand-up,41.0,83.916334,2.046740,0.309758,5278815.0,128751.585366,48901.492595
rotate-outward,40.0,50.935163,1.273379,0.192394,2104242.0,52606.050000,18325.925825
swipe-down,41.0,67.505919,1.646486,0.285907,3789590.0,92429.024390,47715.085493
swipe-left,41.0,65.501331,1.597593,0.278997,3709310.0,90470.975610,36633.244260


In [14]:
for gest, num, tt, at, st, te, ae, se in df.itertuples():
    if gest == "<blank>":
        print(f"|<blank>|-|\SI{{{tt / 60:.2f}}}{{\minute}}|{int(te):,}|")
    else:
        print(f"|{gest}|{int(num)}|\SI{{{int(np.round(tt))}}}{{\second}} ($\mu = \SI{{{at:.1f}}}{{\second}}, \sigma = \SI{{{st:.1f}}}{{\second}}$)|{int(te):,} ($\mu = {int(ae):,}, \sigma = {int(se):,}$)|")

|beckoning|41|\SI{67}{\second} ($\mu = \SI{1.6}{\second}, \sigma = \SI{0.2}{\second}$)|2,949,776 ($\mu = 71,945, \sigma = 29,515$)|
|finger-snap|40|\SI{59}{\second} ($\mu = \SI{1.5}{\second}, \sigma = \SI{0.3}{\second}$)|2,203,881 ($\mu = 55,097, \sigma = 19,777$)|
|ok|40|\SI{63}{\second} ($\mu = \SI{1.6}{\second}, \sigma = \SI{0.2}{\second}$)|3,054,306 ($\mu = 76,357, \sigma = 18,290$)|
|push-hand-down|41|\SI{77}{\second} ($\mu = \SI{1.9}{\second}, \sigma = \SI{0.3}{\second}$)|4,000,518 ($\mu = 97,573, \sigma = 33,154$)|
|push-hand-left|40|\SI{74}{\second} ($\mu = \SI{1.9}{\second}, \sigma = \SI{0.4}{\second}$)|3,859,846 ($\mu = 96,496, \sigma = 32,781$)|
|push-hand-right|39|\SI{76}{\second} ($\mu = \SI{1.9}{\second}, \sigma = \SI{0.3}{\second}$)|5,550,913 ($\mu = 142,331, \sigma = 57,933$)|
|push-hand-up|41|\SI{84}{\second} ($\mu = \SI{2.0}{\second}, \sigma = \SI{0.3}{\second}$)|5,278,815 ($\mu = 128,751, \sigma = 48,901$)|
|rotate-outward|40|\SI{51}{\second} ($\mu = \SI{1.3}{\second

In [13]:
np.sum(list(num_instances.values()))

647